In [1]:
import requests 

In [15]:
def HeaderStr2HeadersDict(headerStr):
    """
    转换F12、等工具获得的请求头字符串成为python字典形式 
    """
    a1 = headerStr.split('\n')[1:-1]
    headers = {}
    for parmer in a1:
        st1 = parmer.split(':')
        key = st1[0]
        value = st1[1]    
        headers[key] =value
    return headers

In [17]:
headerStr= """Accept: */*
Accept-Encoding: identity
Accept-Language: zh-CN,zh;q=0.9
Connection: keep-alive
Host: cn-sccd3-cmcc-v-13.bilivideo.com
Origin: https://www.bilibili.com
Range: bytes=976-1715
Referer: https://www.bilibili.com/video/av90377770/
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: cross-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"""

url = 'https://cn-gslz-cmcc-v-06.bilivideo.com/upgcxcode/63/20/44582063/44582063_da2-1-30280.m4s?expires=1587274500&platform=pc&ssig=Af8xR4zuQ_lpSTRhCReMRw&oi=3085136246&trid=dff7ae52d8114ece92d10063635df270u&nfc=1&nfb=maPYqpoel5MI3qOUX6YpRA==&mid=36871549&logo=80000000'

headers = HeaderStr2HeadersDict(headerStr)

r = requests.get(url,headers)

In [18]:
r.status_code

453

来自Github的一份爬取bilibili用户的代码似乎挺标准化


In [21]:
import requests
from fake_useragent import UserAgent
import json
import queue
import threading
import time
import pymysql
import os
from pathlib import Path
import shutil

ua = UserAgent()


# 多线程
class MyThread(threading.Thread):
    def __init__(self, mid_queue):
        super(MyThread, self).__init__()
        self.mid_queue = mid_queue
        self.cnn = pymysql.connect(host='127.0.0.1', user='root', password='', port=3306, database='bilibili',
                              charset='utf8')
        self.cursor = self.cnn.cursor()
        self.sql = "insert into user_info (mid, name, sex, face, sign, level, birthday, official, archive_view, article_view, likes, follower, following) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

    def run(self):
        while not self.mid_queue.empty():
            dict = self.get_message()
            self.save(dict)

    def get_message(self):
        mid = self.mid_queue.get()
        info_dict = {}

        main_url = f"https://api.bilibili.com/x/space/acc/info?mid={mid}&jsonp=jsonp"
        like_view = f"https://api.bilibili.com/x/space/upstat?mid={mid}&jsonp=jsonp"
        following_follower = f"https://api.bilibili.com/x/relation/stat?vmid={mid}&jsonp=jsonp"

        headers = {
            "Referer": f"https://space.bilibili.com/{mid}",
            "User-Agent": ua.random
        }
        proxies = {
            'http': 'http://113.195.155.211:9999',
            'https': 'http://183.164.239.47:9999'
        }

        response = requests.get(main_url, headers=headers, proxies=proxies).json()

        if response['code'] == -404:
            # code=-404的为被封用户，无信息
            info_dict["id"] = mid
            info_dict["message"] = "该用户已被封号"
        else:
            # main_info
            main_response = response["data"]
            info_dict["mid"] = main_response['mid']
            info_dict["用户名"] = main_response["name"]
            info_dict["性别"] = main_response["sex"]
            info_dict["头像地址"] = main_response["face"]
            info_dict["签名"] = main_response["sign"]
            info_dict["等级"] = main_response["level"]
            info_dict["生日"] = main_response["birthday"]
            info_dict["认证信息"] = main_response["official"]["title"]

            # like&view info
            like_view_response = requests.get(like_view, headers=headers, proxies=proxies).json()["data"]
            info_dict["播放数"] = like_view_response["archive"]["view"]
            info_dict["阅读数"] = like_view_response["article"]["view"]
            info_dict["获赞数"] = like_view_response["likes"]

            # follower&following info
            following_follower_response = requests.get(following_follower, headers=headers, proxies=proxies).json()[
                "data"]
            info_dict["粉丝数"] = following_follower_response["follower"]
            info_dict["关注数"] = following_follower_response["following"]

            print(f'正在爬取:{mid}--{info_dict["用户名"]}信息~~~')

            img = requests.get(info_dict["头像地址"], headers=headers, proxies=proxies).content
            self.face_download(img, info_dict["用户名"], info_dict["头像地址"][-3:])
        return info_dict

    def save(self, dict):
        try:
            self.cursor.execute(self.sql, (dict['mid'], dict['用户名'], dict['性别'], dict['头像地址'], dict['签名'], dict['等级'], dict['生日'], dict['认证信息'], dict['播放数'], dict['阅读数'], dict['获赞数'], dict['粉丝数'], dict['关注数']))
            self.cnn.commit()
        except Exception as error:
            print(error)
            self.cnn.rollback()

    def face_download(self, img, name, extension_name):
        path = os.getcwd()
        path = os.path.join(path, 'img')
        dir = Path(path)
        if not dir.exists():
            os.makedirs(path)
        with open(path + f'\\{name}.{extension_name}', 'wb') as fp:
            fp.write(img)
            fp.close()


def main():
    threadings = []

    mid_queue = queue.Queue()
    mid = {mid_queue.put(str(i)) for i in range(1, 5)}  # 总用户数458241268

    for i in range(10):
        thread = MyThread(mid_queue)
        threadings.append(thread)
        thread.start()

    for i in threadings:
        i.join()


if __name__ == '__main__':
    start = time.time()
    main()
    end = time.time()
    print(f'用时{end - start}s')

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "D:\Anaconda\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "D:\Anaconda\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "D:\Anaconda\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "D:\Anaconda\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "D:\Anaconda\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "D:\Anaconda\lib\http\client.py", line 956, in send
    self.connect()
  File "D:\Anaconda\lib\http\client.py", line 1384, in connect
    super().connect()
  File "D:\Anaconda\lib\http\client.py", line 928, in connect
    (self.host,self.port),

OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([WinError 10061] 由于目标计算机积极拒绝，无法连接。)")

In [5]:
import mysql.connector

In [7]:
conn = mysql.connector.connect(user='root',password='iamyourtf618',database='test')

In [8]:
cursor = conn.cursor()

In [11]:
cursor.execute('create table user (id varchar(20) primary key , name varchar(20))')